### import data and create x and Y

In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [18]:
df = pd.read_csv("house_price_full.csv")

In [9]:
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [19]:
Y = df.pop('price')
x = df

In [20]:
print(x.shape, Y.shape)

(499, 2) (499,)


In [26]:
Y_scaled = np.log(Y)
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled)

In [31]:
x1, x2 = df_scaled.iloc[0]

### pass the inputs x1, x2 to single neuron

In [40]:
# w1 * x1 + w2 * x2 + b = z
# h = activationFunction(z)
w1 = tf.Variable([0.2])
w2 = tf.Variable([0.15])
b = tf.Variable([0.1])

In [43]:
z = b + w1 * x1 + w2 * x2
h = tf.math.sigmoid(z)
h

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.47511354], dtype=float32)>

### Pass the inputs x1 and x2 to multiple neurons in two layers

In [44]:
#input -> layer 1 -> layer 2
# layer1 = [neuron1, neuron2]
# layer2 = [neuron3]

In [51]:
w11 = tf.Variable([0.2])
w12 = tf.Variable([0.15])
b1 = tf.Variable([0.1])

In [54]:
z1 = b1 + w11 * x1 + w12 * x2
h1 =  tf.math.sigmoid(z1)
h1

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.47511354], dtype=float32)>

In [49]:
w21 = tf.Variable([0.5])
w22 = tf.Variable([0.6])
b2 = tf.Variable([0.25])

In [55]:
z2 = b2 + w21 * x1 + w22 * x2
h2 =  tf.math.sigmoid(z2)
h2

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.39686295], dtype=float32)>

In [57]:
w31 = tf.Variable([0.3])
w32 = tf.Variable([0.2])
b3 = tf.Variable([0.4])

In [75]:
z3 = b3 + w31 * h1 + w32 * h2
#h3 =  tf.math.sigmoid(z3)
h3 = z3
h3

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.62190664], dtype=float32)>

In [76]:
y_true = Y_scaled[0]
y_pred = h3.numpy()
0.5*(y_true-y_pred)**2

array([72.38514], dtype=float32)

### Forward pass matrix multiplication

In [ ]:
### W * X + B = Z
### h = activationFunction(Z)

In [ ]:
#input -> layer 1 -> layer 2
# layer1 = [neuron1, neuron2]
# layer2 = [neuron3]

In [ ]:
X = tf.constant([[x1], [x2]])

In [85]:
W1 = tf.Variable([[0.2, 0.15], [0.5, 0.6]])
B1 = tf.Variable([[0.1],[0.25]])
Z1 = tf.matmul(W1, X) + B1
H1 = tf.math.sigmoid(Z1)
H1

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.47511354],
       [0.39686295]], dtype=float32)>

In [88]:
W2 = tf.Variable([[0.3, 0.2]])
B2 = tf.Variable([[0.4]])
Z2 = tf.matmul(W2, H1) + B2
Z2

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.62190664]], dtype=float32)>

In [92]:
y_true = Y_scaled[0]
y_pred = Z2.numpy()
loss = 0.5*(y_true-y_pred)**2
print(loss)

[[72.38514]]


### Random Init Weights: Forward pass matrix multiplication

In [130]:
def get_random_init():
    W1 = tf.Variable( tf.random.uniform((2, 2)) )
    W2 = tf.Variable( tf.random.uniform((1, 2)) )
    B1 = tf.Variable( tf.random.uniform((2, 1)) )
    B2 = tf.Variable( tf.random.uniform((1, 1)) )
    return W1, B1, W2, B2

In [135]:
x = tf.constant([[x1], [x2]], dtype=tf.float32)
y_true = Y_scaled[0]
W1, B1, W2, B2 = get_random_init()

In [132]:
def forward_pass(x, W1, B1, W2, B2):
    H1 = tf.math.sigmoid(tf.matmul(W1, x) + B1)
    Z2 = tf.matmul(W2, H1) + B2
    return Z2

In [136]:
y_pred = forward_pass(x, W1, B1, W2, B2)

In [137]:
#loss
L = 0.5*(y_true - y_pred)**2
print("The MSE error is",L)

The MSE error is tf.Tensor([[67.77809]], shape=(1, 1), dtype=float32)


## Backward propogation

## Part6: Backpropagation

Find the value of x that minimises $y = x^2+4x$

Gradient descent update equation

$x_{new} := x_{old}-\eta\frac{dy}{dx}$

In [139]:
x = tf.Variable([0.0])
lr = eta = 0.1

In [143]:
with tf.GradientTape() as tape:
    y = x**2+4*x
grad = tape.gradient(y, x)

In [144]:
grad.numpy()

array([4.], dtype=float32)

In [145]:
x.assign_sub(lr*grad)##x(new) = x(old)-dy/dx*lr

<tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([-0.4], dtype=float32)>

In [146]:
x.numpy()

array([-0.4], dtype=float32)

### for loop for x 

In [149]:
x = tf.Variable(0.0)
lr = eta = 0.1
for i in range(10):
    with tf.GradientTape() as tape:
        y = x**2+4*x
    grad = tape.gradient(y, x)
    x.assign_sub(lr*grad)
    print(x.numpy())

-0.4
-0.72
-0.9760001
-1.1808001
-1.34464
-1.4757121
-1.5805696
-1.6644557
-1.7315645
-1.7852516


In [168]:
x = tf.constant([[x1], [x2]], dtype=tf.float32)
y_true = Y_scaled[0]
def get_random_init():
    W1 = tf.Variable( tf.random.uniform((2, 2)) )
    W2 = tf.Variable( tf.random.uniform((1, 2)) )
    B1 = tf.Variable( tf.random.uniform((2, 1)) )
    B2 = tf.Variable( tf.random.uniform((1, 1)) )
    return W1, B1, W2, B2
    
def forward_pass(x, W1, B1, W2, B2):
    H1 = tf.math.sigmoid(tf.matmul(W1, x) + B1)
    Z2 = tf.matmul(W2, H1) + B2
    return Z2

In [169]:
w1,b1,w2,b2 = get_random_init()

In [170]:
with tf.GradientTape() as tape:
    y_pred = forward_pass(x, w1, b1, w2, b2)
    loss = 0.5 * (y_true - y_pred) ** 2

In [171]:
gw1, gb1, gw2, gb2 = tape.gradient(loss, [w1, b1, w2, b2])

In [172]:
print(gw1.numpy(), gb1.numpy(), gw2.numpy(), gb2.numpy(), sep="\n")

[[0.9491257  1.6503693 ]
 [0.22500665 0.39124855]]
[[-2.1909761 ]
 [-0.51940876]]
[[-5.326065  -5.9763613]]
[[-11.208692]]


In [173]:
print(w1.numpy(), b1.numpy(), w2.numpy(), b2.numpy(), sep="\n")

[[0.4987179  0.9717995 ]
 [0.68403935 0.7546501 ]]
[[0.848668  ]
 [0.99772525]]
[[0.7838173  0.18617964]]
[[0.973549]]


In [174]:
lr=0.01
print(f"Value of w1 before gradient update is {w1}")
w1.assign_sub(lr*gw1)
b1.assign_sub(lr*gb1)
w2.assign_sub(lr*gw2)
b2.assign_sub(lr*gb2)
print(w1.numpy(), b1.numpy(), w2.numpy(), b2.numpy(), sep="\n")


Value of w1 before gradient update is <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0.4987179 , 0.9717995 ],
       [0.68403935, 0.7546501 ]], dtype=float32)>
[[0.48922664 0.9552958 ]
 [0.6817893  0.7507376 ]]
[[0.87057775]
 [1.0029193 ]]
[[0.8370779  0.24594325]]
[[1.0856359]]
